<a href="https://colab.research.google.com/github/dennistay1981/Resources/blob/main/EDA%20TSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [ ]:
!pip install pmdarima
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

Import, scale, and plot data

In [ ]:
data = pd.read_csv('test.csv', index_col='Time')
data.index = pd.to_datetime(data.index)

scaler = MinMaxScaler()

data['scaled'] = scaler.fit_transform(data['SCL'].values.reshape(-1, 1))

plt.figure(figsize=(12, 6))
plt.plot(data['scaled'])
plt.title('SCL for XXX condition (XXX.csv)')

series = data['scaled']

Compute and plot (P)ACF

In [ ]:
lag_acf = acf(series, nlags=20)
lag_pacf = pacf(series, nlags=20, method='ols')


plt.figure(figsize=(12, 6))
plt.subplot(121)
plot_acf(series, lags=20, ax=plt.gca())
plt.subplot(122)
plot_pacf(series, lags=20, method='ols', ax=plt.gca())
plt.title('(P)ACF for XXX condition (XXX.csv)')
plt.show()

Resample data by seconds, if necessary

In [ ]:
data_seconds = data['scaled'].resample('S').mean()
# Fill NaN values (if any) - choose an appropriate method
data_seconds = data_seconds.fillna(method='ffill') # Forward fill
# Plot resampled data
plt.figure(figsize=(12, 6))
plt.plot(data_seconds)
plt.title('Resampled Data (Seconds)')
plt.show()

series = data_seconds

Auto fit best ARIMA model

In [ ]:
model = auto_arima(series, start_p=0, start_q=0, test='adf', max_p=3, max_q=3, d=None,seasonal=True,
                      start_P=0, D=None, trace=True,  error_action='ignore', suppress_warnings=True,
                      stepwise=True)


print(model.summary())

In [ ]:
# prompt: extract AIC from model

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

# Import libraries
# !pip install pmdarima  # Already installed in the provided code

# ... (rest of the provided code) ...

# Auto fit best ARIMA model
model = auto_arima(series, start_p=0, start_q=0, test='adf', max_p=3, max_q=3, d=None,seasonal=True,
                      start_P=0, D=None, trace=True,  error_action='ignore', suppress_warnings=True,
                      stepwise=True)

# Extract AIC from the model summary
aic = model.aic()
print(f"AIC of the model: {aic}")

Plot data vs. predictions and evaluate model

In [ ]:
predictions = model.predict_in_sample()
#Create axes
x1 =np.arange(0, len(series))
x3= np.arange(0, len(series))

plt.plot(x3,series, label="actual", color='dodgerblue')
plt.plot(x1,predictions, label="predictions",color='orange')
plt.legend()
plt.title('Actual vs. predicted for XXX condition (XXX.csv)')
plt.show()


print("RMSE:", np.sqrt(np.mean((series -predictions) ** 2)))
print("MAPE:", mean_absolute_percentage_error(series, predictions) )
print("R2:", r2_score(series, predictions))
print("AIC:", model.aic())